<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

def get_us_sector_performance():
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response = requests.get(url)
    while response.status_code !=200:
      response = requests.get(url)
    bsoup = BeautifulSoup(response.content,'html.parser')
    
    for i in range(len(bsoup.select('a.heading1'))):
      sector_id = bsoup.select('a.heading1')[i]['id']
      url_sector = f"https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector={sector_id}"
      response = requests.get(url_sector)
      while response.status_code !=200:
        response = requests.get(url)
      content = BeautifulSoup(response.content,'html.parser')
      try:
        output_list.append((content.h1.text,
                          float(str(content.find('span', {'class' : 'negative'})).split('>')[1].split('<')[0][:-1]),
                          str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('>')[1].split('<')[0],
                          float(str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('</span>')[2].split('<span>')[-1][:-1]),
                          url_sector))
      except:
        try:
          output_list.append((content.h1.text,
                          float(str(content.find('span', {'class' : 'positive'})).split('>')[1].split('<')[0][:-1]),
                          str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('>')[1].split('<')[0],
                          float(str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('</span>')[2].split('<span>')[-1][:-1]),
                          url_sector))
        except:
          None
    output_list.sort(key=lambda k: k[-2],reverse=True)
    return output_list

In [ ]:
def get_sector_change_and_market_cap(sector_page_link):
    response = requests.get(sector_page_link)
    while response.status_code !=200:
      response = requests.get(sector_page_link)
    content = BeautifulSoup(response.content,'html.parser')
    try:
        sector_change = float(str(content.find('span', {'class' : 'negative'})).split('>')[1].split('<')[0][:-1])
        sector_market_cap = str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('>')[1].split('<')[0]
        sector_market_weight = float(str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('</span>')[2].split('<span>')[-1][:-1])
    except:
        try:
          sector_change = float(str(content.find('span', {'class' : 'positive'})).split('>')[1].split('<')[0][:-1])
          sector_market_cap = str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('>')[1].split('<')[0]
          sector_market_weight = float(str(re.findall('<span>.*%</span>,',str(content.find_all('span')))).split('</span>')[2].split('<span>')[-1][:-1])
        except:
          None  
    return sector_change,sector_market_cap,sector_market_weight

In [ ]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, '$3.58T', 6.80)
#Note that neither the -0.40, nor the 6.80, end with a % sign

(0.82, '$6.53T', 11.66)

In [ ]:
#Test get_us_sector_performance()
get_us_sector_performance()

[('Information Technology',
  0.6,
  '$11.43T',
  26.75,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  0.46,
  '$6.93T',
  14.87,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Consumer Discretionary',
  0.78,
  '$6.53T',
  11.66,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Communication Services',
  0.17,
  '$4.04T',
  8.08,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Industrials',
  0.21,
  '$4.46T',
  7.83,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Staples',
  -0.56,
  '$4.06T',
  6.96,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30')

In [ ]:
#Running the above cell should return (example: obviously the results will vary over time!)
"""
[('Information Technology',
  2.16,
  '$14.64T',
  28.02,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  0.19,
  '$7.60T',
  13.22,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Consumer Discretionary',
  1.96,
  '$8.32T',
  11.94,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Financials',
  0.63,
  '$8.83T',
  11.32,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Communication Services',
  0.78,
  '$5.74T',
  10.01,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Industrials',
  0.43,
  '$5.58T',
  8.06,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('Consumer Staples',
  -0.16,
  '$4.26T',
  6.27,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
 ('Energy',
  0.12,
  '$3.19T',
  3.27,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
 ('Real Estate',
  0.69,
  '$1.71T',
  2.72,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
 ('Utilities',
  0.55,
  '$1.63T',
  2.6,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
 ('Materials',
  0.25,
  '$2.53T',
  2.58,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15')]
  
"""

In [ ]:
#Print the different sector names from the above output in alpabetical order

In [ ]:
result = get_us_sector_performance()
result.sort()
print(result)

[('Communication Services', 0.27, '$4.04T', 8.08, 'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'), ('Consumer Discretionary', 0.98, '$6.53T', 11.66, 'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'), ('Consumer Staples', -0.57, '$4.06T', 6.96, 'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'), ('Energy', 1.91, '$3.18T', 4.34, 'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'), ('Financials', 0.04, '$6.92T', 10.95, 'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'), ('Health Care', 0.46, '$6.93T', 14.87, 'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'), ('Industrials', 0.27, '$4.46T', 7.83, 'https://eresearch.fidelity.co

In [ ]:
### Find the sector with lowest Market cap by creating a dict of all sectors and their market caps
##    example of the output. {'Materials': '$2.01T', ...}

In [ ]:
result.sort(key=lambda k: k[2])
# We can create a dict:
result_dict = {f'{result[i][0]}': result[i][2] for i in range(len(result))}
print('Full dictionnary',result_dict)
print()
print('Sector with lowest Market cap',{list(result_dict.keys())[0]:result_dict[list(result_dict.keys())[0]]})

Full dictionnary {'Real Estate': '$1.34T', 'Utilities': '$1.60T', 'Materials': '$1.97T', 'Information Technology': '$11.43T', 'Energy': '$3.18T', 'Communication Services': '$4.04T', 'Consumer Staples': '$4.06T', 'Industrials': '$4.46T', 'Consumer Discretionary': '$6.53T', 'Financials': '$6.92T', 'Health Care': '$6.93T'}

Sector with lowest Market cap {'Real Estate': '$1.34T'}


<h3>Hint: How to sort tuples by an arbitrary element?</h3>

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
#Sort by the first element of the tuple
x.sort()
x

[('a', 23.2, 'b'), ('c', 17.4, 'f'), ('d', 29.2, 'z'), ('e', 1.74, 'bb')]

In [ ]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
#Sort by the element at position 1
x.sort(key=lambda k: k[1])
x

[('e', 1.74, 'bb'), ('c', 17.4, 'f'), ('a', 23.2, 'b'), ('d', 29.2, 'z')]